## Importing important libraries and reading the training and testing data

In [45]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import xgboost as xgb
import pickle as pickle

In [46]:
def loadData(text):
    # for reading also binary mode is important
    dbfile = open(text+'.pickle', 'rb')     
    db = pickle.load(dbfile)
    dbfile.close()
    return db
train = loadData('X_train')
test = loadData('X_test')
y_train = loadData('y_train')
y_test = loadData('y_test')
test_df_matrix = loadData('test_df_matrix')
test_df = pd.read_csv("preprocessed_test.csv")
print(train)

FileNotFoundError: [Errno 2] No such file or directory: 'X_train.pickle'

In [ ]:
test_df_matrix.shape

(306122, 178219)

## Doing train test split of the train_df.csv

In [ ]:
# import numpy as np 
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
# train_df = pd.read_csv("preprocessed_train.csv")
# test_df = pd.read_csv("preprocessed_test.csv")

In [ ]:
# test_df

## Vectorization of the dataset

In [ ]:

# tfidf = CountVectorizer()
# # Numericalize the train dataset
# train = tfidf.fit_transform(train_df.question_text.values.astype('U'))
# # Numericalize the test dataset
# # test = tfidf.transform(X_test.values.astype('U'))

# # for test_df
# test_df_matrix = tfidf.transform(test_df.question_text.values.astype('U'))
# # print(test_df_matrix.shape)
# train

In [ ]:
# train

In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(
#     train, train_df.target.values, test_size=0.15, stratify=train_df.target.values)
# X_train.shape

# # test_x = test_df.question_text
# train = X_train
# test = X_test

In [ ]:
# print(train)

## Applying Multinomial Naive Bayes to the model

In [ ]:
model = MultinomialNB()
model.fit(train, y_train)
print("train f1 score:", metrics.f1_score(y_train,model.predict(train)))
print("test f1 score:", metrics.f1_score(y_test,model.predict(test)))
print(metrics.classification_report(y_train,model.predict(train)) )


train f1 score: 0.569139502171121
test f1 score: 0.5177596288600009
              precision    recall  f1-score   support

           0       0.97      0.96      0.97    797411
           1       0.53      0.62      0.57     52589

    accuracy                           0.94    850000
   macro avg       0.75      0.79      0.77    850000
weighted avg       0.95      0.94      0.94    850000



## Applying Logistic Regression to our dataset

In [ ]:
model = LogisticRegression(max_iter=50000,solver='lbfgs')
model.fit(train,y_train)
y_pred = model.predict(train)
print("train f1 score: ",f1_score(y_train,y_pred))
print("test f1 score: ",f1_score(y_test,model.predict(test)))
test_y_pred = model.predict(test_df_matrix)
print(metrics.classification_report(y_train,y_pred) )

train f1 score:  0.5714116966110304
test f1 score:  0.4922717592910627
              precision    recall  f1-score   support

           0       0.97      0.99      0.98    797411
           1       0.75      0.46      0.57     52589

    accuracy                           0.96    850000
   macro avg       0.86      0.73      0.77    850000
weighted avg       0.95      0.96      0.95    850000



### Saving the model and its parameters using pickle

In [ ]:
# # Save the trained model as a pickle string.
# saved_model = pickle.dumps(model)
  
# # Load the pickled model
# model_from_pickle = pickle.loads(saved_model)
  
# # Use the loaded pickled model to make predictions
# # knn_from_pickle.predict(X_test)
# model_from_pickle.fit(train,y_train)
# print(metrics.classification_report(y_train,model_from_pickle.predict(train)) )
# model_from_pickle


### Applying XGBoost on our dataset

In [ ]:
model = xgb.XGBClassifier(learning_rate = 0.9,n_estimators = 1000)
model.fit(train,y_train)
y_pred = model.predict(train)
print("train f1 score: ",f1_score(y_train,y_pred))
print("test f1 score: ",f1_score(y_test,model.predict(test)))
test_y_pred = model.predict(test_df_matrix)
print(metrics.classification_report(y_train,y_pred) )

train f1 score:  0.6435356287854551
test f1 score:  0.5231334769932817
              precision    recall  f1-score   support

           0       0.97      0.99      0.98    797411
           1       0.77      0.55      0.64     52589

    accuracy                           0.96    850000
   macro avg       0.87      0.77      0.81    850000
weighted avg       0.96      0.96      0.96    850000



In [ ]:
model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.9, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

### Creating sample_submission.csv

In [ ]:
Test_DF_TARGET = pd.DataFrame(test_y_pred,columns=['target'])
TEST_DF_QID = pd.DataFrame(test_df ,columns=['qid'])
TEST_DF = pd.concat([TEST_DF_QID, Test_DF_TARGET], axis=1, join='inner')
TEST_DF.to_csv("sample_submission.csv",index=False)